In [28]:
#导入需要的包
import requests
import bs4
import pandas as pd
import re
import time
import pdfkit
import csv
user_agent = {'User-agent': 'Mozilla/5.0'} #Set the user agent.

In [2]:
#baseurl中type=post表示看帖，subType=2表示笔经面经类，order=create表示按发表时间由新到旧排序
baseurl = "https://www.nowcoder.com/search?type=post&subType=2&tagId=0&order=create" 
#采取米哈游+运营作为例子，在牛客网手动搜索感兴趣的主题并复制URL地址即可获得汉字主题的URL编码，空格会被替换为加号
mihayou='%E7%B1%B3%E5%93%88%E6%B8%B8+%E8%BF%90%E8%90%A5'
linkbase='https://www.nowcoder.com'

In [30]:
#创建存储主题帖发表时间，标题和地址的csv文件，并读取为pandas的dataframe
try:
    db=pd.read_csv('mihayou.csv')
except:
    csvfile = open('mihayou.csv', 'w')
    writer = csv.writer(csvfile)
    writer.writerow(['发表时间', '标题', '地址'])
    csvfile.close()
    db=pd.read_csv('mihayou.csv')

In [4]:
#逐页爬取并存储主题帖的标题、地址和发表时间
i=0
while 1:
    i=i+1
    url=baseurl+'&query='+mihayou+'&page='+str(i)
    time.sleep(3)
    result=requests.get(url,headers=user_agent)
    soup=bs4.BeautifulSoup(result.text)
    #如果一共只有4页主题帖，从第五页开始之后的页面就会出现这样一个提示找不到内容的span
    if soup.find('span',{'class':'empty-letter'}):break
    postlist=soup.find('ul',{'class':'common-list'})
    posts=postlist.find_all('li')
    for post in posts:
        title=post.find('a',{'rel':'prefetch'}).text
        title=title.replace('\n','')
        link=linkbase+post.find('a',{'rel':'prefetch'})['href']
        if link in list(db['地址']): continue
        timefield=post.find('p',{'class':'feed-tip'}).text
        publishtime=re.findall('\d{4}-\d{2}-\d{2}',timefield)[0]
        db.loc[len(db)]=[publishtime,title,link]



In [5]:
#将结果写入csv文件
db.to_csv('mihayou.csv')

In [6]:
db.head()

,发表时间,标题,地址
0,2019-10-25,米哈游数据运营一面面经,https://www.nowcoder.com/discuss/324491?type=p...
1,2019-10-23,许愿上岸+米哈游海外社区运营面经（电面）,https://www.nowcoder.com/discuss/322276?type=p...
2,2019-10-23,米哈游玩家运营岗一面面经,https://www.nowcoder.com/discuss/322128?type=p...
3,2019-09-27,米哈游活动运营面经,https://www.nowcoder.com/discuss/287787?type=p...
4,2019-09-20,米哈游活动运营一面面经,https://www.nowcoder.com/discuss/270282?type=p...


In [25]:
#通过刚爬取到的地址列表，使用pdfkit将所有页面合并生成为一个pdf文件，可能会花费较长的时间
urllist=list(db['地址'])

In [26]:
#第一次使用前，请按照提示安装wkhtmltopdf
#https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf
#可以使用brew安装：brew install Caskroom/cask/wkhtmltopdf
pdfkit.from_url(urllist,'mihayou.pdf')
print('Success')

OSError: wkhtmltopdf reported an error:
Loading pages (1/6)
[>                                                           ] 0%[>                                                           ] 0%[>                                                           ] 0%[>                                                           ] 1%[>                                                           ] 1%[>                                                           ] 1%[=>                                                          ] 2%[=>                                                          ] 2%[=>                                                          ] 2%[=>                                                          ] 3%[=>                                                          ] 3%[=>                                                          ] 3%[==>                                                         ] 4%[==>                                                         ] 4%[===>           